In [2]:
import json
import torch
import transformers as tf
import numpy as np

from sklearn.linear_model import LogisticRegression

In [3]:
with open('./data/train.jsonl', 'r') as js_file:
    js_list = list(js_file)
line=js_list[0]
sample=json.loads(line)
sample

{'word': 'place',
 'sentence1': 'Do you want to come over to my place later?',
 'sentence2': 'A political system with no place for the less prominent groups.',
 'idx': 0,
 'label': False,
 'start1': 31,
 'start2': 27,
 'end1': 36,
 'end2': 32,
 'version': 1.1}

In [4]:
'''
Load sentence in to numpy. The symbol '+' here denotes seperation
'''
sen1=np.array([json.loads(i)["sentence1"] for i in js_list])
sen2=np.array([json.loads(i)["sentence2"] for i in js_list])
sep=np.array([' + ']*len(sen1))# sep array
sen_full=np.char.add(sen1,sep)
sen_full=np.char.add(sen_full,sen2)

label=np.array([json.loads(i)["label"] for i in js_list]) #load label

In [5]:
#Load pre trained BERT in the package transformer
model_class, tokenizer_class, pretrained_weights=(tf.BertModel, tf.BertTokenizer, 'bert-base-uncased')
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [6]:
'''
Tokenize two sentences with int
'''
tokenized_sen=np.array([tokenizer.encode(sen1[i],sen2[i],add_special_tokens=True) for i in range(5000)]);

<ipython-input-6-8d07cd27aadc>:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  tokenized_sen=np.array([tokenizer.encode(sen1[i],sen2[i],add_special_tokens=True) for i in range(5000)]);


In [7]:
'''
Padding
Align the length to the max length
'''
def find_max(npa):
    max_len = 0
    for i in npa:
        if len(i) > max_len:
            max_len = len(i)
    return(max_len)
def padding(npa):
    max_len=find_max(npa)
    return (np.array([i + [0]*(max_len-len(i)) for i in npa]))

In [8]:
token_sen_afp=padding(tokenized_sen)
mask = np.where(token_sen_afp != 0, 1, 0)#1 is non padding part 0 o.w
inp = torch.tensor(token_sen_afp).to(torch.int64)#convert to tensor and in64 for the BERT transformer
mask = torch.tensor(mask)

In [10]:
'''
Require at least 4GB RAM
I cost about half an hour to run.
'''


with torch.no_grad():
    outcome = model(inp,mask)
    
features = outcome[0][:,0,:].numpy()
#please neglect idx error here

NameError: name 'idx' is not defined

In [12]:
features.shape

(5000, 768)

In [13]:
lgR = LogisticRegression(max_iter=1000)#here's our train
lgR.fit(features, label)

LogisticRegression(max_iter=1000)

In [24]:
lgR.coef_[0].shape

(768,)

In [14]:
lgR.score(features,label)

0.7614

In [16]:
with open('./data/test.jsonl', 'r') as test_file:
    test_list = list(test_file)
t_line=test_list[0]
sample=json.loads(t_line)
sample

{'word': 'class',
 'sentence1': 'An emerging professional class.',
 'sentence2': 'Apologizing for losing your temper, even though you were badly provoked, showed real class.',
 'idx': 0,
 'label': False,
 'start1': 25,
 'start2': 85,
 'end1': 30,
 'end2': 90,
 'version': 1.1}

In [25]:
tsen1=np.array([json.loads(i)["sentence1"] for i in test_list])
tsen2=np.array([json.loads(i)["sentence2"] for i in test_list])
test_label=np.array([json.loads(i)["label"] for i in test_list]) 

In [18]:
test_tokenized_sen=np.array([tokenizer.encode(tsen1[i],tsen2[i],add_special_tokens=True) for i in range(len(tsen1))]);

<ipython-input-18-c1a2d9aa9ffe>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  test_tokenized_sen=np.array([tokenizer.encode(tsen1[i],tsen2[i],add_special_tokens=True) for i in range(len(tsen1))]);


In [39]:
test_token_sen_afp=padding(test_tokenized_sen)
test_mask = np.where(test_token_sen_afp != 0, 1, 0)
test_inp = torch.tensor(test_token_sen_afp).to(torch.int64)
test_mask = torch.tensor(test_mask)

In [40]:
test_inp.shape

torch.Size([638, 60])

In [41]:
with torch.no_grad():
    test_outcome = model(test_inp,test_mask)

In [42]:
test_features = test_outcome[0][:,0,:].numpy()

In [45]:
lgR.score(test_features,test_label)#with BERT model, no-tunning, the best accuracy is about 60%

0.5940438871473355